In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
from datetime import date, datetime, timedelta, time
import pandas as pd
import seaborn
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import emcee
import triangle
from scipy.stats import multivariate_normal, norm
import pathos.multiprocessing as pathos_mp

In [ ]:
from simlightcurve.solvers import find_rise_t, find_peak
import scipy.optimize as op
from simtransient.models.supernova import Sn1aOpticalEnsemble
import simtransient.utils as stutils
from simtransient import measures, hammer
import simtransient.plot as stplot


In [ ]:
# seaborn.set_context('talk')
seaborn.set_context('poster')
current_palette = seaborn.color_palette()
#Poster
# bigfontsize=40
# labelfontsize=35
# tickfontsize=25
#Paper
bigfontsize=20
labelfontsize=18
tickfontsize=18
plt.rcParams.update({'font.size': bigfontsize,
                     'axes.labelsize':labelfontsize,
                     'xtick.labelsize':tickfontsize,
                     'ytick.labelsize':tickfontsize,
                     'legend.fontsize':tickfontsize,
                     })

In [ ]:
obs_sigma=0.1
detection_thresh = 4.
analysis_thresh = 3.

In [ ]:
ensemble = Sn1aOpticalEnsemble()
gpar_hypers_rv = multivariate_normal(mean=ensemble.gauss_pars.loc['mu'], 
                                     cov=ensemble.gauss_cov)

In [ ]:
ensemble.gauss_pars

In [ ]:
true_gpars = pd.Series(ensemble.gauss_pars.T.mu)
true_gpars[:]= gpar_hypers_rv.rvs()

In [ ]:
true_t0 = 5 #+np.random.random()*5
true_pars = true_gpars.copy()
true_pars['t0']=true_t0
true_pars

In [ ]:
true_curve = ensemble.get_curve(**true_pars)

In [ ]:
tstep = 1.5
sim_epochs = np.arange(start = -30., stop=50, step=tstep)
sim_data = true_curve(sim_epochs)+norm(scale=obs_sigma).rvs(size=len(sim_epochs))

sim_data = pd.Series(data=sim_data, index=sim_epochs)
# sim_data

In [ ]:
threshold = obs_sigma*detection_thresh

print "Thresh:", threshold
detectable = sim_data[sim_data>obs_sigma*detection_thresh]
monitorable = sim_data[sim_data>obs_sigma*analysis_thresh]
first_detection_epoch = detectable.index[0]
usable_data = monitorable[first_detection_epoch:]
# usable_data

In [ ]:
#Detect:
n_data_epochs = 3
obs_data = usable_data.iloc[:n_data_epochs]
# obs_data = usable_data.iloc[::2].iloc[:3]
obs_data

In [ ]:
seaborn.set_style('darkgrid')
seaborn.palplot(current_palette)

In [ ]:
tsteps= np.linspace(-30,50, 1000)
plt.plot(tsteps, true_curve(tsteps),c='g', ls='--',label='True')

plt.scatter(sim_data.index, sim_data,c=current_palette[2])

plt.axhline(obs_sigma*detection_thresh, ls='--', label='Detection')
plt.axhline(obs_sigma*analysis_thresh, ls='-.', label='Analysis')

plt.scatter(usable_data.index, usable_data,color=current_palette[1],s=55)

#Data we'll be using for fitting:
plt.scatter(obs_data.index,obs_data,
            color=current_palette[-1],s=160, lw=3, marker='x',
           label='Fitted data')

# plt.yscale('log')
# plt.axhline(true_amp, ls=':')

# plt.axvline(true_t0, ls='--')
plt.ylim(-1.05*np.abs(np.min(usable_data)),1.05*np.max(usable_data))
plt.legend(loc='best')
plt.gcf().suptitle('Model, observables, detections')
plt


In [ ]:
# detection_t0_offset = find_rise_t(zero_t0_curve, threshold=obs_data.iloc[0], 
#                                      t_min=-3*zero_t0_curve.rise_tau, t_max=1)
# t0_guess = obs_data.index[0]-detection_t0_offset
# print t0_guess, true_t0

In [ ]:
#Initialize a ML fit at the maximum Gaussian priors, and wildly guess at t0
init_model_pars = pd.Series(ensemble.gauss_pars.T.mu, copy=True)
init_model_pars.name="Initial"
init_model_pars['t0']=obs_data.index[0]
init_model_pars

In [ ]:
neg_likelihood = lambda *args: -measures.gauss_lnlikelihood(*args)
results = op.minimize(neg_likelihood, init_model_pars.copy(), args=(ensemble,obs_data, obs_sigma))
max_likelihood_pars = pd.Series(init_model_pars, name='MaxLikelihood', copy=True)
max_likelihood_pars[:]=results.x
print max_likelihood_pars
ml_curve=ensemble.get_curve(**max_likelihood_pars)

In [ ]:
obs_data_min = np.min(obs_data)
obs_data_max = np.max(obs_data)

max_timespan = max_likelihood_pars.decay_tau*10

bounded_t0_lnprior = measures.get_uniform_lnprior(obs_data_min-max_timespan, obs_data_max+max_timespan)
ensemble.t0_lnprior = bounded_t0_lnprior

def lnprob(model_pars, model_ensemble, obs_data, obs_sigma):
    lp = model_ensemble.lnprior(model_pars)
    if not np.isfinite(lp):
        prob = -np.inf
    else:
        prob = lp + measures.gauss_lnlikelihood(model_pars, model_ensemble,obs_data,obs_sigma)
    return prob

In [ ]:
obs_data_min-max_timespan, obs_data_max+max_timespan

In [ ]:
neg_lnprob = lambda *args: -lnprob(*args)
results = op.minimize(neg_lnprob, init_model_pars, args=(ensemble,obs_data, obs_sigma))
# results.x
map_pars = init_model_pars.copy()
map_pars[:]=results.x
map_curve = ensemble.get_curve(**map_pars)
map_pars

In [ ]:
seaborn.set_style('dark')

data_color = current_palette[1]
stplot.curve.graded_errorbar(obs_data, obs_sigma, data_color, alpha=0.8)
plt.plot(tsteps,ml_curve(tsteps), ls='-.',lw=5, label='ML',c=current_palette[0])
plt.plot(tsteps,map_curve(tsteps), ls='-', label='MAP', c=current_palette[0], zorder=1)
plt.plot(tsteps,true_curve(tsteps), ls='--', lw=5,label='true', c=current_palette[-2])

# plt.errorbar(obs_data.index, obs_data, color=data_color)
plt.ylim(0,max(obs_data)+3*obs_sigma)
plt.legend()

In [ ]:
np.atleast_2d(data_color)

In [ ]:
map_pars

In [ ]:
ndim = len(map_pars)  # number of parameters in the model
nwalkers = 50  # number of MCMC walkers
nsteps = 100  # number of MCMC steps to take
# nsteps = 2  # number of MCMC steps to take
# nburn=1
nthreads=4
#for PT
ntemps=10

pool = pathos_mp.Pool(nthreads)

theta_init = map_pars.values

# pos = [starting_point+ 1e-4*np.random.randn(ndim) for i in range(nwalkers)]
# pos = [starting_point+ 1e-4*np.random.randn(ndim) for i in range(nwalkers)]

prop_scale=2
en_nwalkers = nwalkers*2

en_sampler, en_init_ball = hammer.prep_ensemble_sampler(
    lnprob = lnprob,
    init_params = map_pars.values,
    args=(ensemble,obs_data, obs_sigma),
    nwalkers=en_nwalkers,
#     nthreads=nthreads,
    pool=pool,
    a=prop_scale,
    ballsize=1e-3,
    )


pt_sampler,pt_init_ball = hammer.prep_pt_sampler(
    lnprior = ensemble.lnprior, 
    init_params = map_pars.values,
    lnlikelihood=measures.gauss_lnlikelihood,
    lnlikeargs=(ensemble,obs_data, obs_sigma),
    nwalkers=nwalkers,
#     nthreads=nthreads,
    pool=pool,
    a=prop_scale,
    ballsize=1e-3,
    )

In [ ]:
%%capture ensemble_stdout
en_sampler.reset()
with np.errstate(all='ignore'):
    _=en_sampler.run_mcmc(en_init_ball, nsteps)
print "Done"

In [ ]:
# %%capture pt_stdout
# pt_sampler.reset()
# # with np.errstate(over='ignore', under='ignore'):
# with np.errstate(all='ignore'):
#     _=pt_sampler.run_mcmc(pt_init_ball, nsteps)
# print "Done"

In [ ]:
use_pt = False

if not use_pt:
    sampler = en_sampler
    rawchain = en_sampler.chain
else:
    sampler = pt_sampler    
    rawchain = pt_sampler.chain[0]
    
cs, trimmed = hammer.trim_chain(sampler, pt=use_pt)
print cs
print trimmed.shape

In [ ]:
map_pars

In [ ]:
stplot.chain.all_walkers(rawchain, cs, map_pars.index)

In [ ]:
stplot.chain.all_hists(rawchain, cs, map_pars.index,nbins=15)


In [ ]:
figure = triangle.corner(trimmed, truths=true_pars.values, labels=true_pars.keys())

In [ ]:
#Forecast:
t_forecast= 40

In [ ]:
from matplotlib import gridspec
seaborn.set_style('darkgrid')
# seaborn.set_style('white')

stplot.curve.forecast_plot(ensemble, trimmed, tsteps, 
                           t_forecast, 
                           true_curve=true_curve,
                          obs_data=obs_data,
                          obs_sigma=obs_sigma,
                          )
